In [1]:
# %load Run_RFR_on_data.py
#!/usr/bin/env python

# In[5]:


import sys, os
import pandas as pd
import dask
import dask.dataframe as dd
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import KFold ,RandomizedSearchCV ,train_test_split, cross_val_predict , cross_val_score
from sklearn.metrics import mean_absolute_error,mean_squared_error,r2_score,classification_report, confusion_matrix,accuracy_score,matthews_corrcoef
from sklearn.ensemble import RandomForestClassifier,RandomForestRegressor
import matplotlib.pyplot as plt
# import seaborn as sns
import numpy as np
np.random.seed(seed = 101)
import xgboost as xgb; print('XGBoost Version:', xgb.__version__)



XGBoost Version: 0.90


In [2]:

big_df = pd.DataFrame()
all_files = [x for x in os.listdir("synapse_DREAM_challenge/CELL_LINES/") if ".csv" in x and x != "subchallenge_1_template_data.csv"  ]


target_cell_lines = ['AU565', 'EFM19', 'HCC2218', 'LY2', 'MACLS2', 'MDAMB436']


target_genes = ['p.ERK', 'p.Akt.Ser473.', 'p.S6', 'p.HER2', 'p.PLCg2']


for m in all_files:
    #print (m)
    df_temp = dd.read_csv("synapse_DREAM_challenge/CELL_LINES/%s"%m)
    big_df = dd.concat([big_df,df_temp],axis=0)


train_df = big_df[~big_df["cell_line"].isin(target_cell_lines)]

X = train_df.drop(target_genes,axis=1)

y = train_df[target_genes]


In [4]:
big_df.columns

Index(['treatment', 'cell_line', 'time', 'cellID', 'fileID', 'b.CATENIN',
       'cleavedCas', 'CyclinB', 'GAPDH', 'IdU', 'Ki.67', 'p.4EBP1',
       'p.Akt.Ser473.', 'p.AKT.Thr308.', 'p.AMPK', 'p.BTK', 'p.CREB', 'p.ERK',
       'p.FAK', 'p.GSK3b', 'p.H3', 'p.HER2', 'p.JNK', 'p.MAP2K3', 'p.MAPKAPK2',
       'p.MEK', 'p.MKK3.MKK6', 'p.MKK4', 'p.NFkB', 'p.p38', 'p.p53',
       'p.p90RSK', 'p.PDPK1', 'p.PLCg2', 'p.RB', 'p.S6', 'p.S6K', 'p.SMAD23',
       'p.SRC', 'p.STAT1', 'p.STAT3', 'p.STAT5'],
      dtype='object')

In [5]:
my_target_ditionary = {'ERK':'p.ERK', 'AKT':'p.Akt.Ser473.', 'S6':'p.S6', 'HER':'p.HER2', 'PLCG2':'p.PLCg2'}

#from dask_ml.xgboost import XGBRegressor
#base_model = RandomForestRegressor( n_estimators = 1000, random_state = 101,n_jobs=-1,verbose=1)


In [6]:
print ("cat4egorize")

X = X.categorize(columns=["treatment"])

print ("dummies")

my_dummies = dd.get_dummies(X["treatment"])


X= X.drop(['treatment', 'cell_line', 'time', 'cellID', 'fileID'],axis=1)


cat4egorize
dummies


In [7]:
# for m in X.columns :
#     if len(X[X[m].isna()]) != 0:
#         print (m,len(X[X[m].isna()]))
    #for m in X.columns :
#    X[m].fillna((X[m].mean().compute())

In [8]:
# for m in y.columns :
#     if len(y[y[m].isna()]) != 0:
#         print (m,len(y[y[m].isna()]))

In [9]:
# len(y)

In [10]:
# 20459760-2248286

In [11]:
print ("filling NA")
y = y.compute()
# y['p.PLCg2_fill'] = y['p.PLCg2'].fillna(y['p.PLCg2'].mean().compute()  )
# y['p.HER2_fill'] = y['p.HER2'].fillna(y['p.HER2'].mean().compute()  )

# # In[ ]:



# #for m in X.columns :
# #    X[m].fillna((X[m].mean().compute()))


# # In[ ]:


for m in y.columns :
    y[m].fillna(y[m].mean() , inplace=True )
# X.fillna(0)
# y.fillna(0)


filling NA


In [12]:
# y = y.drop(['p.PLCg2','p.HER2'],axis=1)
# for m in y.columns :
#     if len(y[y[m].isna()]) != 0:
#         print (m,len(y[y[m].isna()]))

In [13]:
y.columns

Index(['p.ERK', 'p.Akt.Ser473.', 'p.S6', 'p.HER2', 'p.PLCg2'], dtype='object')

In [12]:
#base_model = xgb.XGBRegressor(verbosity=2,n_jobs=-1,objective='reg:squarederror')
base_model = RandomForestRegressor(verbose=2,n_jobs=-1,random_state=101,n_estimators=1000)

In [13]:
# # instantiate params
# params = {}

# # general params
# general_params = {'silent': 1}
# params.update(general_params)

# # booster params
# n_gpus = 1
# booster_params = {}

# if n_gpus != 0:
#     booster_params['tree_method'] = 'gpu_hist'
#     booster_params['n_gpus'] = n_gpus
# params.update(booster_params)

# # learning task params
# learning_task_params = {'eval_metric': 'rmse', 'objective': 'reg:tweedie'}
# params.update(learning_task_params)
# print(params)

In [14]:
print("Scaler")
scaler = StandardScaler()

scaler.fit(X)

X_scaled = scaler.transform(X)

Scaler


In [15]:
# type(X_scaled)
X_scaled[:100]

array([[-0.03506847,  1.67391572, -0.84339623, ...,  0.63902696,
        -0.28732705, -0.04874762],
       [-1.15458435, -0.36247582, -0.69648374, ..., -2.53343822,
        -0.95625421, -1.797144  ],
       [-1.06581749, -1.24644988, -0.95014624, ..., -0.6328717 ,
        -1.08873047,  0.10586511],
       ...,
       [-1.15382871,  0.97053614, -0.95014624, ...,  0.20882815,
        -0.06610757, -0.26242267],
       [-1.15458435, -1.24644988, -0.81029226, ..., -0.48433516,
        -1.08873047, -0.49845811],
       [ 1.4811292 ,  1.34956188, -0.54617928, ...,  0.65249188,
         0.53978631, -0.39644473]])

In [16]:
# type(y)
y[:100]

,p.ERK,p.Akt.Ser473.,p.S6,p.HER2,p.PLCg2
0,1.985215,3.537355,5.96652,4.988744,2.185986
1,1.923010,0.550685,3.08669,2.420064,0.986227
2,0.650769,3.308237,4.75636,5.003612,1.653599
3,4.394234,2.917066,5.69939,4.794311,2.457316
4,0.962249,0.550685,3.33093,3.733031,2.208131
5,2.347849,1.669070,5.07211,3.701532,1.379199
6,2.571471,1.774273,2.72194,3.033706,0.986227
7,3.520213,3.875774,4.82766,3.802993,2.036127
8,1.730348,3.030022,2.39632,4.183928,3.299247
9,3.237769,3.838371,5.17159,4.003189,3.197022


In [ ]:
my_results = []
my_dict = {}
for key in my_target_ditionary.keys():
    print (key,my_target_ditionary[key])
    #X_train, X_test, y_train, y_test = train_test_split(X_scaled[:100] , y[my_target_ditionary[key]][:100], test_size=0.33, random_state=101)
    X_train, X_test, y_train, y_test = train_test_split(X_scaled , y[my_target_ditionary[key]], test_size=0.33, random_state=101)
    #print (type(y_train))
    base_model.fit(X_train,y_train )
    #dtrain = xgb.DMatrix(X_train, label=y_train)
    #dvalidation = xgb.DMatrix(X_test, label=y_test)
    #dtrain = xgb.DMatrix(X_train)
    #dvalidation = xgb.DMatrix(X_test)
    
# model training settings
    #evallist = [(dvalidation, 'validation'), (dtrain, 'train')]
    #num_round = 2
    #base_model.fit(params, dtrain,y_train )
    #base_model.train(params, dtrain, num_round, evallist)
    predictions = base_model.predict(X_test ) 
    #predictions = base_model.predict(dvalidation)
    print ("R^2:",r2_score(y_test, predictions))
    print ("MAE:",mean_absolute_error(y_test, predictions))
    print ("MSE:",mean_squared_error(y_test, predictions))
    r=r2_score(y_test, predictions)
    mae=mean_absolute_error(y_test, predictions)
    mse=mean_squared_error(y_test, predictions)
    print ("------------------------------------------")
    my_results.append((key,r,mae,mse))
    
    
    error = predictions - y_test
    my_dict[my_target_ditionary[key]] = { "y_test":y_test , "predictions":predictions , "error":error }
#     error_plot = plt.hist(error, bins = 25)
#     error_plot.xlabel("Prediction Error [MAE]")
#     error_plot.ylabel("Count")
#     error_plot.savefig("basemodel_error_bin_%s.png"%(key),format="png")
    

#     plt.scatter(y_test, predictions)
#     plt.xlabel('True Values [MAE]')
#     plt.ylabel('Predictions [MAE]')
#     plt.axis('equal')
#     plt.axis('square')
#     plt.xlim([0,plt.xlim()[1]])
#     plt.ylim([0,plt.ylim()[1]])
#     _ = plt.plot([-100, 100], [-100, 100])
#     plt.savefig("MAE_base_model_%s.png"%(key),format="png")



ERK p.ERK


[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 8 concurrent workers.


building tree 1 of 1000
building tree 2 of 1000
building tree 3 of 1000
building tree 4 of 1000
building tree 5 of 1000
building tree 6 of 1000
building tree 7 of 1000
building tree 8 of 1000
building tree 9 of 1000
building tree 10 of 1000
building tree 11 of 1000
building tree 12 of 1000
building tree 13 of 1000
building tree 14 of 1000
building tree 15 of 1000
building tree 16 of 1000
building tree 17 of 1000
building tree 18 of 1000
building tree 19 of 1000
building tree 20 of 1000
building tree 21 of 1000
building tree 22 of 1000
building tree 23 of 1000
building tree 24 of 1000
building tree 25 of 1000
building tree 26 of 1000
building tree 27 of 1000
building tree 28 of 1000
building tree 29 of 1000
building tree 30 of 1000
building tree 31 of 1000
building tree 32 of 1000
building tree 33 of 1000


[Parallel(n_jobs=-1)]: Done  25 tasks      | elapsed: 102.4min


building tree 34 of 1000
building tree 35 of 1000
building tree 36 of 1000
building tree 37 of 1000
building tree 38 of 1000
building tree 39 of 1000
building tree 40 of 1000
building tree 41 of 1000
building tree 42 of 1000
building tree 43 of 1000
building tree 44 of 1000
building tree 45 of 1000
building tree 46 of 1000
building tree 47 of 1000
building tree 48 of 1000
building tree 49 of 1000
building tree 50 of 1000
building tree 51 of 1000
building tree 52 of 1000
building tree 53 of 1000
building tree 54 of 1000
building tree 55 of 1000
building tree 56 of 1000
building tree 57 of 1000
building tree 58 of 1000
building tree 59 of 1000
building tree 60 of 1000
building tree 61 of 1000
building tree 62 of 1000
building tree 63 of 1000
building tree 64 of 1000
building tree 65 of 1000
building tree 66 of 1000
building tree 67 of 1000
building tree 68 of 1000
building tree 69 of 1000
building tree 70 of 1000
building tree 71 of 1000
building tree 72 of 1000
building tree 73 of 1000


In [ ]:
result_summary = pd.DataFrame(my_results)


In [ ]:
result_summary.columns = ["Target","R^2","MAE","MSE"]

In [ ]:
print (result_summary)


In [ ]:
for key in my_dict:
    print (key)
    plt.figure()
    plt.hist(my_dict[key]["error"], bins = 25)
    plt.xlabel("Prediction Error [MAE]")
    _ = plt.ylabel("Count")
    plt.savefig("basemodel_error_bin_%s.png"%(key),format="png")
    
    

In [ ]:
for key in my_dict:
    print (key)
    plt.figure()
    plt.scatter(my_dict[key]["y_test"], my_dict[key]["predictions"])
    plt.xlabel('True Values [MAE]')
    plt.ylabel('Predictions [MAE]')
    plt.axis('equal')
    plt.axis('square')
    plt.xlim([0,plt.xlim()[1]])
    plt.ylim([0,plt.ylim()[1]])
    _ = plt.plot([-100, 100], [-100, 100])
    plt.savefig("MAE_base_model_%s.png"%(key),format="png")

In [ ]:
# plt.scatter(y_test, predictions)
# plt.xlabel('True Values [MAE]')
# plt.ylabel('Predictions [MAE]')
# plt.axis('equal')
# plt.axis('square')
# plt.xlim([0,plt.xlim()[1]])
# plt.ylim([0,plt.ylim()[1]])
# _ = plt.plot([-100, 100], [-100, 100])

In [ ]:
# error_plot = plt.hist(error, bins = 25)
# plt.xlabel("Prediction Error [MAE]")
# plt.ylabel("Count")

In [1]:
!ls synapse_DREAM_challenge/CELL_LINES/ | wc -l 


65


In [15]:
base_model = xgb.XGBRFRegressor(n_jobs=-1,n_estimators=8,)

In [16]:
X_train, X_test, y_train, y_test = train_test_split(X_scaled , y["p.ERK"], test_size=0.33, random_state=101)

In [17]:
d_train = xgb.DMatrix(X_train,y_train)

/home/didier/anaconda3/envs/dask-tutorial/lib/python3.7/site-packages/xgboost/core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \


In [18]:
d_test = xgb.DMatrix(X_test,y_test)

In [31]:
type(d_test)


xgboost.core.DMatrix

In [27]:
# param = {'silent':1, 'objective':'reg:lineal', 'booster':'gbtree', 'base_score':3}
param = {'silent':1,  'booster':'gbtree', 'base_score':3}

In [28]:
# for gamma regression, we need to set the objective to 'reg:gamma', it also suggests
# to set the base_score to a value between 1 to 5 if the number of iteration is small

In [29]:
# the rest of settings are the same
watchlist = [(d_test, 'eval'), (d_train, 'train')]
num_round = 30

In [30]:
bst = xgb.train(param, d_train, num_round, watchlist)

[0]	eval-rmse:0.970925	train-rmse:0.97049
[1]	eval-rmse:0.872155	train-rmse:0.871691
[2]	eval-rmse:0.816031	train-rmse:0.815587
[3]	eval-rmse:0.784241	train-rmse:0.783816
[4]	eval-rmse:0.765851	train-rmse:0.7654
[5]	eval-rmse:0.754224	train-rmse:0.753746
[6]	eval-rmse:0.746945	train-rmse:0.746423
[7]	eval-rmse:0.741515	train-rmse:0.74097
[8]	eval-rmse:0.737801	train-rmse:0.737218
[9]	eval-rmse:0.734354	train-rmse:0.733738
[10]	eval-rmse:0.731935	train-rmse:0.731309
[11]	eval-rmse:0.729572	train-rmse:0.728933
[12]	eval-rmse:0.727683	train-rmse:0.727031
[13]	eval-rmse:0.72577	train-rmse:0.725105
[14]	eval-rmse:0.724828	train-rmse:0.724156
[15]	eval-rmse:0.723497	train-rmse:0.722816
[16]	eval-rmse:0.721807	train-rmse:0.721108
[17]	eval-rmse:0.720677	train-rmse:0.719966
[18]	eval-rmse:0.719643	train-rmse:0.718908
[19]	eval-rmse:0.718642	train-rmse:0.717904
[20]	eval-rmse:0.717804	train-rmse:0.717052
[21]	eval-rmse:0.716527	train-rmse:0.715771
[22]	eval-rmse:0.715699	train-rmse:0.714937
[23